In [1]:
import os
import numpy as np
import torch
from torch import nn, optim
from torchsummary import summary
import torch.nn.functional as F
from pennylane import *
from classical_unet import *
from train import *
from unitary import * 
from subdataset import dataset_images_train
from QCUnet import * 
from QCUBottleNeck import * 
import pandas as pd

w:\summer school\Methane leaks\base research\code\Methane-Plume-Segmentation-main\QCUnet-methane-leaks\train.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


cpu
cpu


In [2]:
len(dataset_images_train)

200

## Benchmarking QCUnet models

In [3]:
##  
"""
 widad => Archi1, Archi3, 
 rihab => Archi2
 radia => Archi4
"""

archi_num ='Archi1'

def QCModels_Benchmarking(archi_num ='Archi1'): 
    """
        [True, False, False] ==> working (Archi1)
        [True, True, False] ==> working  (Archi2)
        [False, False, True] ==> working (Archi3)
        [True, False, True] ==> working  (Archi4)
        [True, True, True] ===> working => we do just one experiment with the best params of the previous ones
    """
    experimets_results_path = f'results\\df_exepriments_{archi_num}.csv'
    if os.path.exists(experimets_results_path):
        df_experiments = pd.read_csv(experimets_results_path)
    else :
        df_experiments= pd.DataFrame(columns = [
                                                'archi',
                                                'model_name', 
                                                'epochs', 
                                                'batch_size',
                                                'bands', 
                                                'lr', 
                                                'unitary',  
                                                "opt", 
                                                "loss",                                          
                                                "train_loss", 
                                                "val_loss", 
                                                "train_iou",
                                                "train_acc", 
                                                "metrics_by_epoch"
                                                ])
        df_experiments.to_csv(experimets_results_path)
    # fixed 
    image_size = 90
    bs = 10
    ep = 5

    # varying []
    bandss = [ [12,13], [1,2,3,4,5,6,7,8,9,10,11,12,13]] 
    lr = [0.001,0.01 , 0.05]
    lr = [0.001,0.01]
    
    # Unitaries = [None, U_TTN, U_5, U_6, U_9, U_13, U_14, U_15, U_SO4, U_SU4]
    # Unitaries = [None, U_TTN, U_5]
    Unitaries = [None]
    architectures = {'Archi1' : [True, False, False], 
                     'Archi2': [True, True, False], 
                     'Archi3' : [False, False, True], 
                     'Archi4' : [True, False, True],  
                     'Archi5': [True, True, True]   
                    }
    
    for bands in bandss: 
        n_channels = len(bands)
        for unitary in Unitaries:
            quantum_layers= architectures[archi_num]
            if quantum_layers == [False, False, True]:
                model = UNetWithQBottleNeck(n_channels=n_channels, n_classes=1, unitary=unitary)
            else:
                model = QCUNet(n_channels=n_channels, quantum_layers= quantum_layers ,n_classes=1, 
                            unitary=unitary)
                
            for learning_rate in lr:                 
                # initialize loss function
                loss = nn.BCEWithLogitsLoss()

                # initialize optimizer
                opt = optim.Adam(model.parameters(),lr=learning_rate)

                # run training
                trained_model, model_metrics, overall_metrics, model_name = train_model(
                                            model, ep, opt, 
                                            loss, bs, bands= bands, 
                                            path_savemodel = f'./mod/quantum/{archi_num}/',
                                            lr= learning_rate,  
                                            train_size=200, 
                                            test_size=50, 
                                            TrainImgfiles = dataset_images_train
                                            )
                row= {
                        'archi': archi_num, 
                        'model_name': model_name, 
                        'epochs': ep, 
                        'batch_size': bs,
                        'bands': bands, 
                        'lr': learning_rate, 
                        'unitary': unitary, 
                        "opt" : 'Adam', 
                        "loss" : 'BCEWithLogitsLoss', 
                        "train_loss": overall_metrics["train_loss"], 
                        "val_loss": overall_metrics["val_loss"], 
                        "train_iou" : overall_metrics["train_iou"],
                        "train_acc": overall_metrics["train_acc"] , 
                        "metrics_by_epoch": model_metrics
                }
                
                df_experiments.loc[len(df_experiments)] = row 
                df_experiments.to_csv(experimets_results_path, index=False)
                
QCModels_Benchmarking(archi_num = archi_num)

params size torch.Size([1, 4])
(200,)
(200,)
(199,)
(199,)
08_47_21, 53
epoch 0
batch  0
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.7138, grad_fn=<DivBackward0>)
Train/Iou 0.123118475
Train/Acc tensor(0.6994)
batch  1
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.7043, grad_fn=<DivBackward0>)
Train/Iou 0.0678243
Train/Acc tensor(0.7232)
batch  2
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.6994, grad_fn=<DivBackward0>)
Train/Iou 0.04840521
Train/Acc tensor(0.7239)
batch  3
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.6909, grad_fn=<DivBackward0>)
Train/Iou 0.04326719
Train/Acc tensor(0.7310)
batch  4
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.6855, grad_fn=<DivBackward0>)
Train/Iou 0.04626053
Train/Acc tensor(0.7312)
batch  5
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.6804, grad_fn=<DivBackward0>)
Train/Iou 0.047727313
Train/Acc tensor(0.7399)
batch  6
x6 shape torch.Size([10, 512, 2, 2])
Train/Loss tensor(0.6

## Benchmarking Classical Unet

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

print('heeellooo')
# set seed
set_all_seeds(21)

# model
bands = [1,2,3,4,5,6,7,8,9,10,11,12,13]
model = UNet(n_channels=13, n_classes=1)
model.to(device)

# trainning parameters
ep = 10
lr = 0.001
bs = 32

# setup tensorboard writer

# initialize loss function
loss = nn.BCEWithLogitsLoss()

# initialize optimizer
opt = optim.Adam(model.parameters(),lr=lr)

# run training
# trained_model = train_model(model, ep, opt, loss, bs, bands= bands, lr= lr)


heeellooo
params size torch.Size([1, 4])
params size torch.Size([1, 4])
x6 shape torch.Size([2, 512, 2, 2])
x6_flatten shape: torch.Size([2, 512, 4])
QCNN output shape: torch.Size([2, 512, 4])
qcnn_output reshaped: torch.Size([2, 512, 2, 2])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantConv-1            [-1, 4, 90, 90]               0
            Conv2d-2            [-1, 9, 90, 90]             333
       BatchNorm2d-3            [-1, 9, 90, 90]              18
              ReLU-4            [-1, 9, 90, 90]               0
       QDoubleConv-5            [-1, 9, 90, 90]               0
         MaxPool2d-6            [-1, 9, 45, 45]               0
            Conv2d-7           [-1, 64, 45, 45]           5,248
       BatchNorm2d-8           [-1, 64, 45, 45]             128
              ReLU-9           [-1, 64, 45, 45]               0
           Conv2d-10           [-1, 64, 45, 45]      